In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamd

In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

b'{"access_token":"BQC4rOOwnZHB2m-f2biefGLR7gfEvhlFclqOjbcSQjeGMjYxxO73vcoArKs1jsaz6ftDJYezRI-MXZSZKJY","token_type":"Bearer","expires_in":3600,"scope":""}'


In [4]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

[{'acousticness': 0.0311,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'danceability': 0.904,
  'duration_ms': 226864,
  'energy': 0.813,
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'instrumentalness': 0.00697,
  'key': 4,
  'liveness': 0.0471,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'tempo': 125.461,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'type': 'audio_features',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'valence': 0.81}]

In [5]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

---
**Note**

We could construct a model which determines if a user will like a certain playlist based on comparing their playlists and liked playlists to the audio features of our playlist. Train model on users playlists and likes, classify playlist as 0 or 1 if the user will like the playlist.

I believe our final model will look something like this: https://developer.spotify.com/console/get-recommendations/

In [8]:
songs = pd.read_pickle("pickles/songs_30k_dropped.pkl")
songs.head()

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
53,1,0.000032,https://api.spotify.com/v1/audio-analysis/1MYY...,0.343,70294,0.975,1MYYt7h6amcrauCOoso3Gx,0.991,1,0.0515,-2.502,0,0.1350,119.883,4,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,audio_features,spotify:track:1MYYt7h6amcrauCOoso3Gx,0.133
54,2,0.000023,https://api.spotify.com/v1/audio-analysis/3x2m...,0.414,65306,0.959,3x2mJ2bjCIU70NrH49CtYR,0.933,7,0.3680,-4.299,0,0.0378,145.911,4,https://api.spotify.com/v1/tracks/3x2mJ2bjCIU7...,audio_features,spotify:track:3x2mJ2bjCIU70NrH49CtYR,0.589
55,3,0.053400,https://api.spotify.com/v1/audio-analysis/1Pm3...,0.522,108532,0.205,1Pm3fq1SC6lUlNVBGZi3Em,0.235,2,0.0985,-7.986,1,0.0376,103.868,4,https://api.spotify.com/v1/tracks/1Pm3fq1SC6lU...,audio_features,spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,0.353
59,7,0.201000,https://api.spotify.com/v1/audio-analysis/7dkb...,0.493,226000,0.969,7dkbEHIMLoeuG4zXGmzhEH,0.655,2,0.2470,-3.282,0,0.0463,170.581,4,https://api.spotify.com/v1/tracks/7dkbEHIMLoeu...,audio_features,spotify:track:7dkbEHIMLoeuG4zXGmzhEH,0.340
88,36,0.007740,https://api.spotify.com/v1/audio-analysis/0hBb...,0.293,658987,0.787,0hBby0yygBY1u3m6tSpZgC,0.496,5,0.3920,-8.841,1,0.0438,165.307,4,https://api.spotify.com/v1/tracks/0hBby0yygBY1...,audio_features,spotify:track:0hBby0yygBY1u3m6tSpZgC,0.138


In [9]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    for col in cols: 
        x_scaled = MinMaxScaler().fit_transform(df_old[col].reshape(-1,1))
        df_new[col] = x_scaled
    return(df_new)
        
songs2 = scaler(cols, songs, songs2)
songs2.head()

/Users/williamdrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


,level_0,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,53,1,0.000032,https://api.spotify.com/v1/audio-analysis/1MYY...,0.346465,70294,0.975,1MYYt7h6amcrauCOoso3Gx,0.991,1,0.0515,-2.502,0,0.139607,0.483471,4,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,audio_features,spotify:track:1MYYt7h6amcrauCOoso3Gx,0.133
1,54,2,0.000023,https://api.spotify.com/v1/audio-analysis/3x2m...,0.418182,65306,0.959,3x2mJ2bjCIU70NrH49CtYR,0.933,7,0.3680,-4.299,0,0.039090,0.588439,4,https://api.spotify.com/v1/tracks/3x2mJ2bjCIU7...,audio_features,spotify:track:3x2mJ2bjCIU70NrH49CtYR,0.589
2,55,3,0.053614,https://api.spotify.com/v1/audio-analysis/1Pm3...,0.527273,108532,0.205,1Pm3fq1SC6lUlNVBGZi3Em,0.235,2,0.0985,-7.986,1,0.038883,0.418885,4,https://api.spotify.com/v1/tracks/1Pm3fq1SC6lU...,audio_features,spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,0.353
3,59,7,0.201807,https://api.spotify.com/v1/audio-analysis/7dkb...,0.497980,226000,0.969,7dkbEHIMLoeuG4zXGmzhEH,0.655,2,0.2470,-3.282,0,0.047880,0.687929,4,https://api.spotify.com/v1/tracks/7dkbEHIMLoeu...,audio_features,spotify:track:7dkbEHIMLoeuG4zXGmzhEH,0.340
4,88,36,0.007771,https://api.spotify.com/v1/audio-analysis/0hBb...,0.295960,658987,0.787,0hBby0yygBY1u3m6tSpZgC,0.496,5,0.3920,-8.841,1,0.045295,0.666660,4,https://api.spotify.com/v1/tracks/0hBby0yygBY1...,audio_features,spotify:track:0hBby0yygBY1u3m6tSpZgC,0.138


In [11]:
song1 = songs2.iloc[np.where(songs.id == '1MYYt7h6amcrauCOoso3Gx')]
song1

,level_0,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,53,1,0.000032,https://api.spotify.com/v1/audio-analysis/1MYY...,0.346465,70294,0.975,1MYYt7h6amcrauCOoso3Gx,0.991,1,0.0515,-2.502,0,0.139607,0.483471,4,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,audio_features,spotify:track:1MYYt7h6amcrauCOoso3Gx,0.133


In [12]:
def distance(idkey, songs):
    song = songs.iloc[np.where(songs.id == idkey)]
    dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                 + (np.subtract(songs.danceability, song.danceability))**2
                 + (np.subtract(songs.energy, song.energy))**2
                 + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                 + (np.subtract(songs.liveness, song.liveness))**2
                 + (np.subtract(songs.speechiness, song.speechiness))**2
                 + (np.subtract(songs.tempo, song.tempo))**2)
        
    return dist

In [13]:
top10_dist = distance('1MYYt7h6amcrauCOoso3Gx', songs2).sort_values()[0:10]

In [14]:
top10_dist.index

Int64Index([0, 91651, 90717, 29581, 21644, 20278, 132056, 20410, 166847,
            132065],
           dtype='int64')

In [15]:
top10 = songs2.iloc[top10_dist.index]

In [16]:
hrefs = top10['id']
hrefs

0         1MYYt7h6amcrauCOoso3Gx
91651     5ZvH28ktf1tLulZzctK3nj
90717     1416zt0Z35YVGt97dXMbxu
29581     5mzEmhsKmDmIsSK6qfn6z6
21644     4EPyA9RnqEFZUQYMfne7d3
20278     38fEzdnGpKOUaYcpRJtVph
132056    3JNgVcrrkGwg1jFHMfu4n7
20410     4GeuGxSl1IiRzgxSLWAFzT
166847    1ILheWuypHpVuTWiZ0PTKA
132065    4DJWXpvDOR2ZV3wDNNKl9N
Name: id, dtype: object

In [17]:
for ref in top10['id']:
    track = sp.track(ref)
    print(track['name'])
    print('by')
    for artist in track['artists']:
        print(artist['name'])
    print('-----------------------------------')

Libera Me From Hell (Tengen Toppa Gurren Lagann)
by
Daniel Tidwell
-----------------------------------
Lalassu Xul
by
Rings of Saturn
-----------------------------------
Right Next Door To Hell
by
Guns N' Roses
-----------------------------------
El borrego
by
Café Tacvba
-----------------------------------
Intoarcerea Amurgului
by
Negura Bunget
-----------------------------------
I Could Never Be Your Man
by
Bass Drum of Death
-----------------------------------
Habitual Infamy
by
Suffocation
-----------------------------------
Angered Kids
by
Kid Cudi
-----------------------------------
City in the Sky
by
Scale The Summit
-----------------------------------
The Invoking
by
Suffocation
-----------------------------------
